<a href="https://colab.research.google.com/github/Krumpu/Homework_ICA/blob/main/HW_3_ICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
url = 'https://raw.githubusercontent.com/Krumpu/Homework_ICA/main/Data-Melbourne_F.csv'
df = pd.read_csv(url)
df = df.drop(columns=['BOD','month', 'day','year'])
df
df_y = df[['COD']].copy()
df_x = df.drop(columns=['COD'])

treino_x, teste_x, treino_y, teste_y = train_test_split(df_x,df_y,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)

assimetria = df.drop('COD', axis=1).skew()
colunas_assimetricas =  ['VV', 'SLP', 'PP', 'avg_inflow']

for colunas in colunas_assimetricas:
  treino_x[colunas] = np.log1p(treino_x[colunas])
  teste_x[colunas] = np.log1p(teste_x[colunas])

In [3]:
mediana = treino_y['COD'].median()


y_train_cls = (treino_y['COD'] > mediana).astype(int).values
y_test_cls = (teste_y['COD'] > mediana).astype(int).values
print(f"Mediana usada para corte: {mediana}")
print(f"Classes no Treino: {np.bincount(y_train_cls)}")

Mediana usada para corte: 840.0
Classes no Treino: [560 545]


In [6]:
#Normalização

mean = treino_x.mean()
std = treino_x.std()

X_train_norm = ((treino_x - mean) / std).values
X_test_norm = ((teste_x - mean) / std).values

X_train_bias = np.c_[np.ones((len(X_train_norm), 1)), X_train_norm]
X_test_bias = np.c_[np.ones((len(X_test_norm), 1)), X_test_norm]

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def treinar_regressao_logistica(X, y, learning_rate=0.01, epochs=1000):
    m = len(y)
    theta = np.zeros(X.shape[1])
    cost_history = []

    for _ in range(epochs):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        theta -= learning_rate * gradient
    return theta

def prever_logistica(X, theta, threshold=0.5):
    probabilidades = sigmoid(np.dot(X, theta))
    return (probabilidades >= threshold).astype(int)